In [ ]:
import requests as req 
from bs4 import BeautifulSoup as bs
import re
import json
import time
import os

In [ ]:
API_KEY = []

with open("API_KEY", "r") as file:
    API_KEY = file.read().split("\n")

In [ ]:
links = [
    #1
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/movie/"},
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/movie3/"},
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/series/"},
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/series2/"},
    {"url": "https://dls.iran-gamecenter-host.com/DonyayeSerial/series3/"},
    #2
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/movie/"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/movies/"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/movies2/"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/movies3/"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/movies4/"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/series/"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/series2/"},
    {"url": "https://dls2.iran-gamecenter-host.com/DonyayeSerial/series3/"},
    #4
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/movies/"},
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/movie3/"},
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/series/"},
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/series2/"},
    {"url": "https://dls4.iran-gamecenter-host.com/DonyayeSerial/series3/"},
    #5
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/series/"},
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/series2/"},
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/series3/"},
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/movies/"},
    {"url": "https://dls5.iran-gamecenter-host.com/DonyayeSerial/movie3/"},
    #6
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/movie/"},
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/movie3/"},
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/movies/"},
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/series/"},
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/series2/"},
    {"url": "https://dls6.iran-gamecenter-host.com/DonyayeSerial/series3/"},
    #7
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/movies/"},
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/movie3/"},
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/series2/"},
    {"url": "https://dls7.iran-gamecenter-host.com/DonyayeSerial/series3/"},
    #8
    {"url": "https://dls8.iran-gamecenter-host.com/DonyayeSerial/movie3/"},
    {"url": "https://dls8.iran-gamecenter-host.com/DonyayeSerial/movie4/"},
    {"url": "https://dls8.iran-gamecenter-host.com/DonyayeSerial/series2/"},
    {"url": "https://dls8.iran-gamecenter-host.com/DonyayeSerial/series4/"},
]

In [ ]:
available = 0
for key in API_KEY:
    if key != "X":
        available += 1
print(f"{available} of {len(API_KEY)} key(s) are avaiable")

In [ ]:
def read_urls(url):
    while True:
        try:
            r = req.get(url)
            break
        except:
            print("Connection failed. trying again in 5 seconds ...")
            time.sleep(5)
        
    soup = bs(r.text, "html.parser")

    file_items = soup.select("td.n > a")

    found_urls = []
    for item in file_items:
        href = item.get("href")
        if href != "../":
            found_urls.append(href)

    return found_urls

In [ ]:
def imdb_data(id, key):
    while True:
        try:
            r = req.get(f"http://www.omdbapi.com/?i={id}&apikey={key}")
            break
        except:
            print("Connection failed. trying again in 5 seconds ...")
            time.sleep(5)
    return r.json()

imdb_recognized = {}

def get_name(string, percentage):
    if string.startswith("tt"):
        try:
            name = imdb_recognized[string]
        except:
            caught = False
            for key in API_KEY:
                if key == "X":
                    continue
                data = imdb_data(string, key)
                try:
                    name = data["Title"]
                    caught = True
                    break
                except:
                    if (key == API_KEY[len(API_KEY)-1]):
                        new_key = input("please enter another ApiKey: ")
                        API_KEY.append(new_key)
                    API_KEY[API_KEY.index(key)] = "X"
                    print("Loading another ApiKey ...")
            print(f"[{percentage}] {name}")
            imdb_recognized[string] = name
    else:
        name = string.replace(".", " ")
    return name

def get_title(url):
    return re.findall(r"((movie|series)[s|\d]?\/(\d\d\d\d\/)?(\w|\.)+)", url)[0][0].split("/")[-1]

In [ ]:
def get_quality(url):
    try:
        return " ".join(re.findall(r"(\.\d{3,4}p\.?.*(Sub|Dub))", url)[0][0].split(".")[1:-1])
    except:
        return "Unknown Quality"
    
def get_season(url):
    try:
        season = re.findall(r"S\d+E\d+", url)[0].split("E")[0].replace("S", "")
    except:
        season = 0
    return season

def get_episode(url):
    try:
        episode = re.findall(r"S\d+E\d+", url)[0].split("E")[1]
    except:
        episode = 0
    return episode

def sub_or_dubbed(url):
    try:
        return re.findall(r"(dub|sub|trailer)", url.lower())[0]
    except:
        return "sub"

def series_or_movie(url):
    return re.findall(r"(series|movie)", url.lower())[0]

def get_datacenter_info(url):
    return "-".join(re.findall(r"(dls\d?).*((movie|serie)s?\d?)", url)[0][0:-1])

In [ ]:
def create_name_if_not_exists(som, name):
    try:
        if films[som][name]:
            pass
    except:
        films[som][name] = {}
def create_sob_if_not_exists(som, name, sob):
    try:
        if films[som][name][sob]:
            pass
    except:
        films[som][name][sob] = {}
def create_quality_if_not_exists(som, name, sob, quality):
    try:
        if films[som][name][sob][quality]:
            pass
    except:
        films[som][name][sob][quality] = {}
def create_season_if_not_exists(som, name, sob, quality, season):
    try:
        if films[som][name][sob][quality][season]:
            pass
    except:
        films[som][name][sob][quality][season] = {}

def create_episode_if_not_exists(som, name, sob, quality, season, episode):
    try:
        if films[som][name][sob][quality][season][episode]:
            pass
    except:
        films[som][name][sob][quality][season][episode] = []

def create_all_if_not_exist(som, name, sob, quality, season, episode):
    create_name_if_not_exists(som, name)
    create_sob_if_not_exists(som, name, sob)
    create_quality_if_not_exists(som, name, sob, quality)
    create_season_if_not_exists(som, name, sob, quality, season)
    create_episode_if_not_exists(som, name, sob, quality, season, episode)    

def classify_url(url, percentage):
    som = series_or_movie(url) # som stands for series or movie
    name = get_name(get_title(url), percentage)
    sob = sub_or_dubbed(url)  # sob stands for sub or dubbed
    quality = get_quality(url)
    season = get_season(url)
    episode = get_episode(url)
    
    create_all_if_not_exist(som, name, sob, quality, season, episode)
    
    films[som][name][sob][quality][season][episode].append(url)

In [ ]:
def is_visited(url):
    try:
        return visited[url]
    except:
        return False

# DFS on Website!

counter = 0
visited = {}
extracted = []

def dfs(url):
    global counter
    counter += 1
    print(f"[{counter}] {url}")
    items = read_urls(url)
    visited[url] = True
    for item in items:
        if item.endswith((".mkv", "mp4", ".srt", ".ssa", ".mka", ".mp3", ".zip", ".rar", ".jpg", ".jpeg", ".png")):
            extracted.append(url + item)
            continue
        new_url = url + item
        if (not is_visited(new_url)):
            dfs(new_url)

In [ ]:
def save_data(films, file_name):
    json_output = json.dumps(films)
    with open(file_name, "w") as file:
        file.write(json_output)

In [ ]:
link_counter = 0
for link in links:
    link_counter += 1
    print(f"Scrapping {link["url"]} ({link_counter} of {len(links)})")
    dfs(link["url"])
    save_data(extracted, get_datacenter_info(link["url"]) + ".json")
    extracted = []
    visited = {}
    counter = 0
print("Scrapping Done.")

In [ ]:
films = {
    "movie": {},
    "series": {}
}

if os.path.exists("temp_final.json"):
    with open("temp_final.json", "r") as file:
        films = json.loads(file.read())

In [ ]:
# merging json files and classifying them
counter = 0
for link in links:
    counter += 1
    print(f"{counter} of {len(links)}")
    file_name = get_datacenter_info(link["url"])
    with open(file_name + ".json", "r") as file:
        saved_data = json.loads(file.read())
        url_counter = 0
        for url in saved_data:
            url_counter += 1
            percentage = str(round(url_counter / len(saved_data), 2) * 100) + "%"
            classify_url(url, percentage)
        save_data(films, "temp_final.json")
print("Done.")

print("File saved.")

In [ ]:
os.rename("temp_final.json", "final.json")